# First Agent

In [1]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai.chat_models.base import ChatOpenAI
llm = ChatOpenAI(temperature=0.1)

In [2]:
prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

In [ ]:
# 5,273.38‬

In [3]:
llm.invoke(prompt)

AIMessage(content='= $4923.38', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 59, 'total_tokens': 65, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-ChwXS7ihle4sfRN7oHe1B5DtFtOod', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--4d972f71-d7e3-4859-b139-cb3e5c9a3904-0', usage_metadata={'input_tokens': 59, 'output_tokens': 6, 'total_tokens': 65, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# ★LLM 은 산술연산을 할수 없다!!
# LLM은 '산술연산'을 수행하지 않습니다.  이런 계산은 AI보다 '계산기'가 더 잘한다!
# LLM 은 text 를 생성해내는 겁니다. 문장 시퀀스의 다음 token 이 무엇인지 통계적으로 추측하는 겁니다.

# 이러한 LLM 의 오류를 잡기위해선 agent 를 제공해주어야 한다.
# 그리고 agent 를 위한 tool(들) 을 만들고, agent 가 tool 을 선택해서 실행하는 것이다.


# Agent 사용

In [5]:
from langchain.agents.initialize import initialize_agent

https://python.langchain.com/api_reference/langchain/agents/langchain.agents.initialize.initialize_agent.html#initialize-agent

```python
langchain.agents.initialize.initialize_agent(
    tools: Sequence[BaseTool],   # tools 들 전달
    llm: BaseLanguageModel,      # llm 전달
    agent: AgentType | None = None,   # AgentType 전달
    callback_manager: BaseCallbackManager | None = None,
    agent_path: str | None = None,
    agent_kwargs: dict | None = None,
    *,
    tags: Sequence[str] | None = None,
    **kwargs: Any,
)
```

In [6]:
from langchain.agents.agent_types import AgentType

In [7]:
# 간단한 tool 준비
# 간단한 함수

def plus(a, b):
    return a + b

In [8]:
# 이 함수를 우리 agent 가 실제로 사용할수 있는 tool 로 변환해서 전달해주어야 한다
#   여기선 우린 StructuredTool 를 사용해보겠다.


In [9]:
from langchain_core.tools.structured import StructuredTool

In [10]:
# agent 생성
agent = initialize_agent(
    tools = [
        StructuredTool.from_function(
            func=plus,   # 함수를 전달 -> tool 변환
            name="Sum Calculator",   # tool 이름
            description="Use this to perform sums of two numbers. This tool take two arguments, both  should be numbers.",
        ),
    ],
    llm = llm,
    agent = AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26184\702377185.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
# StructuredTool => tool 인 plus(a, b) 가 '여러개의 input' 을 갖는 툴

In [ ]:
"""
AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION

https://python.langchain.com/api_reference/langchain/agents/langchain.agents.agent_types.AgentType.html#langchain.agents.agent_types.AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION

An zero-shot react agent optimized for chat models.
This agent is capable of invoking tools that have multiple inputs.  <- tool 이 여러개의 input 을 받는 AgentType!

- ZERO SHOT 은 나중에 배워보자.
- REACT <- react.js 와 관계 없다.   react 논문기반이라는 뜻.
"""
None

In [13]:
# agent 호출
prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.01 + $552.01 + $76.16 + $29.12"
agent.invoke(prompt)

# 



> Entering new AgentExecutor chain...
Thought: I can use the Sum Calculator tool to find the total cost of all these amounts added together.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 355.39,
    "b": 924.87
  }
}
```
Observation: 1280.26
Thought:I made a mistake by only summing the first two numbers. Let's continue adding the rest of the amounts to get the total cost.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 1280.26,
    "b": 721.2
  }
}
```
Observation: 2001.46
Thought:I need to continue adding the remaining amounts to the total.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 2001.46,
    "b": 1940.29
  }
}
```
Observation: 3941.75
Thought:I have calculated the sum of all the amounts correctly so far. Let's continue adding the remaining amounts to get the final total.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 3941.75,
    "b": 573.63
  }
}
```
Observation: 4515.38


{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.01 + $552.01 + $76.16 + $29.12',
 'output': 'The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.01 + $552.01 + $76.16 + $29.12 is $5273.40.'}